In [ ]:
!pip install eemont
!pip install rasterio
!pip install earthpy


In [ ]:
import ee
import eemont
import geemap as geemap
import os
import math
import rasterio
from rasterio.windows import Window
import folium
import earthpy.plot as ep
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imageio


In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-andreagonnelli06')


In [ ]:
# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

In [ ]:
Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map

In [ ]:
rectangle = Map.draw_last_feature
rectangle.getInfo()



In [ ]:
coordinates = [12.771073, 42.151186,12.834922, 42.189736]
def clippa (image):

  return image.clip(ee.Geometry.Rectangle(coordinates))

def scala (image):
  return image.divide(10000)
region = ee.Geometry.Rectangle(coordinates)

collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(ee.Geometry.Rectangle(coordinates)) \
    .filterDate('2021-09-09', '2021-09-13') \
    .map(clippa)\
    .map(scala)


count = collection.size()
print('Count: ', count.getInfo())


In [ ]:
visualizationR = {'min': 0.0,
                 'max': 0.3,
                  'strech': 1,
                 'bands': ['B4','B3','B3']}

listOfImages = collection.toList(collection.size())

In [ ]:
Image = collection.first()
#Image = Image.scaleAndOffset()
#Map.addLayer(Image, visualization, 'RGB' + str(i))
Image = Image.select(['B2', 'B3', 'B4'])
Map.addLayer(Image, visualizationR, 'R' )
Map.centerObject(collection, 8)
Map

In [ ]:
cd /content

In [ ]:
out_dir = os.path.expanduser('/content/sentinel2_nerola')
#geemap.ee_export_image_collection(collection, out_dir=out_dir)
geemap.ee_export_image(Image, filename='image1.tif', scale=10, region=coordinates, file_per_band=False)
#geemap.ee_export_image(Image2, filename='image2.tif', scale=10, region=coordinates, file_per_band=False)
#geemap.ee_export_image(Image3, filename='image3.tif', scale=10, region=coordinates, file_per_band=False)


In [ ]:
def generate_tiles(image_file, output_file, area_str, size=32):
    """Generates 64 x 64 polygon tiles.

    Args:
      image_file (str): Image file path (.tif)
      output_file (str): Output file path (.geojson)
      area_str (str): Name of the region
      size(int): Window size

    Returns:
      GeoPandas DataFrame: Contains 64 x 64 polygon tiles
    """

    # Open the raster image using rasterio
    raster = rio.open(image_file)
    width, height = raster.shape

    # Create a dictionary which will contain our 64 x 64 px polygon tiles
    # Later we'll convert this dict into a GeoPandas DataFrame.
    geo_dict = { 'id' : [], 'geometry' : []}
    index = 0

    # Do a sliding window across the raster image
    with tqdm(total=width*height) as pbar:
      for w in range(0, width, size):
          for h in range(0, height, size):
              # Create a Window of your desired size
              window = rio.windows.Window(h, w, size, size)
              # Get the georeferenced window bounds
              bbox = rio.windows.bounds(window, raster.transform)
              # Create a shapely geometry from the bounding box
              bbox = box(*bbox)

              # Create a unique id for each geometry
              uid = '{}-{}'.format(area_str.lower().replace(' ', '_'), index)

              # Update dictionary
              geo_dict['id'].append(uid)
              geo_dict['geometry'].append(bbox)

              index += 1
              pbar.update(size*size)

    # Cast dictionary as a GeoPandas DataFrame
    results = gpd.GeoDataFrame(pd.DataFrame(geo_dict))
    # Set CRS to EPSG:4326
    results.crs = {'init' :'epsg:4326'}
    # Save file as GeoJSON
    results.to_file(output_file, driver="GeoJSON")

    raster.close()
    return results

In [ ]:
def create_tiles(input_path, output_folder, tile_width=32, tile_height=32):
    with rasterio.open(input_path) as src:
        # Calcola il numero di righe e colonne necessarie per la griglia
        cols = math.ceil(src.width / tile_width)
        rows = math.ceil(src.height / tile_height)

        # Itera su ogni riga e colonna per ritagliare le tile
        for row in range(rows):
            for col in range(cols):
                # Calcola la finestra di ritaglio per la tile corrente
                xoff = col * tile_width
                yoff = row * tile_height
                width = min(tile_width, src.width - xoff)
                height = min(tile_height, src.height - yoff)
                window = Window(xoff, yoff, width, height)

                # Leggi i dati dalla finestra e crea una nuova immagine
                tile_data = src.read(window=window)
                transform = src.window_transform(window)
                tile_meta = src.meta.copy()
                tile_meta.update({
                    'width': width,
                    'height': height,
                    'transform': transform
                })

                # Salva la tile come un nuovo file TIFF
                tile_path = os.path.join(output_folder, f'tile_{row}_{col}.tif')
                with rasterio.open(tile_path, 'w', **tile_meta) as dst:
                    dst.write(tile_data)

# Specifica il percorso del file di input e la cartella di output per le tile
input_path = '/content/image1.tif'
output_folder = 'tiles_output1'

# Crea la cartella di output se non esiste
os.makedirs(output_folder, exist_ok=True)

# Crea le tile
create_tiles(input_path, output_folder)

print("Ritaglio delle tile completato.")


Ritaglio delle tile completato.


In [ ]:
# Percorso del repository contenente le immagini
repository_path = "/content/tiles_output1"

# Elenco dei file nel repository
file_names = os.listdir(repository_path)

# Contatore per tenere traccia del numero di immagini elaborate
image_counter = 0

# Iterazione attraverso i file nel repository
for file_name in file_names:
    # Verifica se il file è un file TIFF
    if file_name.endswith('.tif'):
        # Percorso completo del file
        file_path = os.path.join(repository_path, file_name)

        # Apertura del file TIFF con rasterio
        with rasterio.open(file_path) as src:
            # Lettura dei canali R, G, B
            image_vis = []
            for x in [3, 2, 1]:
                image_vis.append(src.read(x))
            image_vis = np.stack(image_vis)

        # Trasposizione per ottenere l'immagine nel formato corretto
        image = np.transpose(image_vis, (1, 2, 0))

        # Verifica e normalizzazione del tipo di dati
        if image.dtype != np.uint8:
            image = ((image*255)).astype(np.uint8)

        # Verifica se le dimensioni dell'immagine sono corrette (32x32x3)
        if image.shape == (32, 32, 3):
            # Salvataggio dell'immagine come file PNG
            output_file_path = f"/content/png_nerola/image{image_counter}.png"
            imageio.imwrite(output_file_path, image)

            # Incremento del contatore delle immagini elaborate
            image_counter += 1
        else:
            print(f"Attenzione: l'immagine {file_name} non ha le dimensioni corrette (32x32x3) e non verrà salvata.")

print(f"Sono state elaborate e salvate {image_counter} immagini.")

# Visualizzazione dell'ultima immagine elaborata
image_path = f"/content/png_nerola/image{image_counter - 1}.png"
image = plt.imread(image_path)
plt.imshow(image)
plt.show()


In [ ]:
cd /content

In [ ]:
!git clone https://github.com/allenai/satlas-super-resolution.git

In [ ]:
cd satlas-super-resolution/

In [ ]:
!pip install -r requirements.txt

In [ ]:
!apt-get install -y python-gdal gdal-bin libgdal-dev

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir experiments

In [ ]:
!cp '/content/drive/MyDrive/esrgan_1S2 (2).pth' '/content/satlas-super-resolution/experiments'

In [ ]:
!wget https://pub-25c498004d1e4d4c8da69b2c05676836.r2.dev/super_resolution_test_set_2024-01-09.tar

In [ ]:
import tarfile

# Specify the path to your tar file
tar_file_path = '/content/satlas-super-resolution/super_resolution_test_set_2024-01-09.tar'

# Specify the directory where you want to extract the contents
extract_dir = '/content/satlas-super-resolution'

# Open the tar file
with tarfile.open(tar_file_path, 'r') as tar:
    # Extract all the contents
    tar.extractall(extract_dir)

In [ ]:
import os
import shutil

# Cartella originale contenente le immagini
original_directory = "/content/png_nerola"

# Nuova cartella in cui verranno create le nuove sottodirectory
new_directory = "/content/satlas-super-resolution/test_set/png_nerola"

# Creare la nuova cartella se non esiste già
os.makedirs(new_directory, exist_ok=True)

# Ottenere la lista dei nomi dei file nella cartella originale
file_names = os.listdir(original_directory)

# Contatore per tenere traccia del numero di sottodirectory create
num_subdirectories_created = 0

# Iterare attraverso i file nella cartella originale
for file_name in file_names:
    # Verifica se il file è un'immagine
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tif', '.tiff')):
        # Creare una nuova sottodirectory per l'immagine nella nuova cartella
        image_subdirectory = os.path.join(new_directory, os.path.splitext(file_name)[0])
        os.makedirs(image_subdirectory, exist_ok=True)

        # Copiare l'immagine nella nuova sottodirectory
        shutil.copy(os.path.join(original_directory, file_name), image_subdirectory)
        image_path = os.path.join(image_subdirectory, file_name)


        # Aggiornare il contatore delle sottodirectory create
        num_subdirectories_created += 1

print(f"Sono state create {num_subdirectories_created} nuove sottodirectory nella cartella {new_directory}.")


Sono state create 208 nuove sottodirectory nella cartella /content/satlas-super-resolution/test_set/png_nerola.


In [ ]:
!python -m ssr.infer -opt ssr/options/infer_example.yml


In [ ]:
# Load the PNG image
img = mpimg.imread('/content/satlas-super-resolution/testing_infer/177/lr.png')  # Replace 'path_to_your_image.png' with the path to your PNG file

# Display the image
plt.imshow(img)
plt.axis('off')  # Turn off axis
plt.show()

In [ ]:
# Load the PNG image
img = mpimg.imread('/content/satlas-super-resolution/testing_infer/177/sr.png')  # Replace 'path_to_your_image.png' with the path to your PNG file

# Display the image
plt.imshow(img)
plt.axis('off')  # Turn off axis
plt.show()

In [ ]:
# Load the PNG image
img = mpimg.imread('/content/satlas-super-resolution/testing_infer/85/lr.png')  # Replace 'path_to_your_image.png' with the path to your PNG file

# Display the image
plt.imshow(img)
plt.axis('off')  # Turn off axis
plt.show()

In [ ]:
# Load the PNG image
img = mpimg.imread('/content/satlas-super-resolution/testing_infer/85/sr.png')  # Replace 'path_to_your_image.png' with the path to your PNG file

# Display the image
plt.imshow(img)
plt.axis('off')  # Turn off axis
plt.show()

In [ ]:
image_path = f"/content/satlas-super-resolution/test_set/png_nerola/image50/image50.png"
image = plt.imread(image_path)
plt.imshow(image)
plt.show()
